In [2]:
import nltk
nltk.download('stopwords')
nltk.download(['punkt', 'wordnet'])
nltk.download('omw-1.4')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ai2318\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ai2318\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ai2318\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ai2318\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
from sqlalchemy import create_engine
import pandas as pd
import sqlite3
import numpy as np


In [4]:
# load data from database
engine = create_engine(
    'sqlite:///H:\\disaster-response-pipeline\\data\\disaster_record.db')

#read table and separate X and Y features
df = pd.read_sql_table('disaster_table', engine)

X = df.iloc[:, 1].values
y = df.iloc[:,5:40].values

In [5]:
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'],
      dtype=object)

In [6]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import re
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report


stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

#Tokenize message
def tokenize(text):
    #remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]"," ", text.lower())

    #tokenize text
    tokens = word_tokenize(text)


    #lemmatize and remove stopwords
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y)


In [15]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])


In [ ]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)


In [ ]:
from sklearn.metrics import classification_report
labels = list(df.iloc[:,4:40].columns)

for index, label in enumerate(labels):
    classification = classification_report(y_test[:,index], y_pred[:,index]);
    print(label,"\n",classification)

In [2]:
#get parameters for pipeline
pipeline.get_params().keys()


NameError: name 'Pipeline' is not defined

In [ ]:
#create gridsearch to improve params
from sklearn.model_selection import GridSearchCV
parameters = {
    'clf__estimator__max_depth': [2, 10, 50]
}

cv = GridSearchCV(pipeline, param_grid=parameters)


In [ ]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)


In [ ]:
cv.best_params_


In [ ]:
for index, label in enumerate(labels):
    classification = classification_report(y_test[:,index], y_pred[:,index]);
    print(label,"\n",classification)